# Elektron WebSocket API MRN Example with Python

## Machine Readable News Overview

Refinitiv Machine Readable News (MRN) is an advanced service for automating the consumption and systematic analysis of news. It delivers deep historical news archives, ultra-low latency structured news and news analytics directly to your applications. This enables algorithms to exploit the power of news to seize opportunities, capitalize on market inefficiencies and manage event risk.

MRN aims for replacing the legacy News 2K (N2_UBMS and N2_STORY).

### MRN Data behavior
MRN is published over Elektron using an Open Message Model (OMM) envelope in News Text Analytics domain messages. The Real-time News content set is made available over MRN_STORY RIC. The content data is contained in a FRAGMENT field that has been compressed, and potentially fragmented across multiple messages, in order to reduce bandwidth and message size.

A FRAGMENT field has a different data type based on a connection type:
* RSSL connection (ESDK [C++](https://developers.refinitiv.com/elektron/elektron-sdk-cc)/[Java](https://developers.refinitiv.com/elektron/elektron-sdk-java)): BUFFER type
* WebSocket connection: Base64 ascii string

The data goes through the following series of transformations:

1. The core content data is a UTF-8 JSON string
2. This JSON string is compressed using gzip
3. The compressed JSON is split into a number of fragments (BUFFER or Base64 ascii string) which each fit into a single update message
4. The data fragments are added to an update message as the FRAGMENT field value in a FieldList envelope


<img src="images/mrn_process.png"/>

Therefore, in order to parse the core content data, the application will need to reverse this process. The WebSocket application also need to convert a received Base64 string in a FRAGMENT field to bytes data before further process this field.

### MRN Data model

Five fields, as well as the RIC itself, are necessary to determine whether the entire item has been received in its various fragments and how to concatenate the fragments to reconstruct the item:
* MRN_SRC: identifier of the scoring/processing system that published the FRAGMENT
* GUID: a globally unique identifier for the data item. All messages for this data item will have the same GUID values.
* FRAGMENT: compressed data item fragment, itself
* TOT_SIZE: total size in bytes of the fragmented data
* FRAG_NUM: sequence number of fragments within a data item. This is set to 1 for the first fragment of each item published and is incremented for each subsequent fragment for the same item.

A single MRN data item publication is uniquely identified by the combination of RIC, MRN_SRC, and GUID.

#### Fragmentation
For a given RIC-MRN_SRC-GUID combination, when a data item requires only a single message, then TOT_SIZE will equal the number of bytes in the FRAGMENT and FRAG_NUM will be 1.

When multiple messages are required, then the data item can be deemed as fully received once the sum of the number of bytes of each FRAGMENT equals TOT_SUM. The consumer will also observe that all FRAG_NUM range from 1 to the number of the fragment, with no intermediate integers skipped. In other words, a data item transmitted over three messages will contain FRAG_NUM values of 1, 2 and 3.

#### Compression
The FRAGMENT field is compressed with gzip compression, thus requiring the consumer to decompress to reveal the JSON plain-text data in that FID.

When an MRN data item is sent in multiple messages, all the messages must be received and their FRAGMENTs concatenated before being decompressed. In other words, the FRAGMENTs should not be decompressed independently of each other.

The decompressed output is encoded in UTF-8 and formatted as JSON.

Please see a full documentation of this example application in [this article](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).

If you are not familiar with MRN concept, please visit the following resources which will give you a full explanation of the MRN data model and implementation logic:
* [Webinar Recording: Introduction to Machine Readable News](https://developers.refinitiv.com/news#news-accordion-nid-12045)
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).

In [1]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install requests and websocket-client packages in a current Jupyter kernal\n

import sys

# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [2]:

import time
import getopt
import socket
import json
import websocket
import threading
from threading import Thread, Event
import base64
import zlib
import requests

### EDP Credentials

Please put in your EDP credentials and client ID here.

In [ ]:
# EDP Credentials

user = ''
password = ''
client_id = ''

In [4]:
# EDP connection variables

token = ''
token_refresh = ''
token_api_endpoint = 'https://api.refinitiv.com/auth/oauth2/v1/token'
hostname = 'apac-3.pricing.streaming.edp.thomsonreuters.com'
port = '443'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [5]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False
counter = 0
sleep_for_seconds = 10

# keeps decompress news JSON messaage
_news_envelopes = []
_news_messages = []

### EDP Token retrival

In order to connect to ERT on Cloud, you need to retrieve token from the Token API first. This part of the code will take care of then token retrieval for you.

In [12]:
# EDP Token Retrieval

def get_token():
    print('Retrieve Token')
    data = {
        'client_id': client_id,
        'grant_type': 'password',
        'password': password,
        'scope': 'trapi',
        'takeExclusiveSignOnControl': True,
        'username': user,
    }
    #print(data)
    r = requests.post(token_api_endpoint,
        data=data,
        verify=True)
    auth_json = r.json()
    #print(json.dumps(auth_json, sort_keys=True, indent=2, separators=(',', ':')))
    response = {
        'token': '',
        'error': '',
        'refresh': ''
    }
    if ('access_token' in auth_json):
        response['token'] = auth_json['access_token']
        response['refresh'] = auth_json['refresh_token']
    else:
        response['error'] = auth_json['error_description']
    return response

# EDP Token Refresh

def refresh_token(refresh):
    print('Refresh Token')
    print(refresh)
    refresh = refresh.encode('ascii')
    
    data = {
        'client_id': client_id,
        'grant_type': 'refresh_token',
        'refresh_token': refresh,
        'username': user,
    }
    #print(data)
    r = requests.post(token_api_endpoint,
        data=data,
        verify=True)
    auth_json = r.json()
    print(json.dumps(auth_json, sort_keys=True, indent=2, separators=(',', ':')))
    response = {
        'token': '',
        'error': '',
        'refresh': ''
    }
    if ('access_token' in auth_json):
        response['token'] = auth_json['access_token']
        response['refresh'] = auth_json['refresh_token']
    else:
        response['error'] = auth_json['error_description']
    return response

# EDP Login

def login(ws, refresh):
    # Get token
    if refresh != None:
        token_result = refresh_token(refresh)
    else:
        token_result = get_token()

    if (token_result['token'] != ''):
        # Success
        global token
        token = token_result['token']
        global token_refresh
        token_refresh = token_result['refresh']
    else:
        # Failed
        print(token_result['error'])
        sys.exit(2)
    
    if (ws != None):
        send_login_request(ws, True)

### MRN Process Code

The MRN data can be subscribed with the *NewsTextAnalytics* domain and MRN-specific RIC name as following:
- *MRN_STORY*: Real-time News
- *MRN_TRNA*: News Analytics: Company and C&E assets
- *MRN_TRNA_DOC*: News Analytics: Macroeconomic News & events
- *MRN_TRSI*: News Sentiment Indices

 

In [7]:
# MRN variables

mrn_domain = 'NewsTextAnalytics'
mrn_item = 'MRN_STORY'

def send_mrn_request(ws):
    """ Create and send MRN request """
    mrn_req_json = {
        'ID': 2,
        "Domain": mrn_domain,
        'Key': {
            'Name': mrn_item
        }
    }

    ws.send(json.dumps(mrn_req_json))
    print("SENT:")
    print(json.dumps(mrn_req_json, sort_keys=True, indent=2, separators=(',', ':')))

### Initial Refresh Message
The Initial Refresh response does not contain any NTA data, all the fields related to news item and fragment are empty or 0. It contains only the relevant feed related or other static Fields. 

The application can just print out each incoming field data in a console for informational purpose or just ignore it.

In [8]:
# Process FieldList, Refresh and Status messages.

def decodeFieldList(fieldList_dict):
    for key, value in fieldList_dict.items():
        print("Name = %s: Value = %s" % (key, value))

def processRefresh(ws, message_json):

    print("RECEIVED: Refresh Message")
    decodeFieldList(message_json["Fields"])

def processStatus(ws, message_json):  # process incoming status message
    print("RECEIVED: Status Message")
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

### MRN News Update messages Process Code

The updates contain only fields related to the item and the fragment. They do not contain any of the static or per-feed fields. The updates are not cached or conflated.

#### First Update
The first update contains all the fields related to the item and the first fragment, subsequent updates only contain the fields relating to the fragment they contain. The FRAG_NUM FID is set to 1 for the first Update of each item and is incremented in each subsequent Update for that item. This allows you to you to detect a missing fragment (and ensure correct order of the fragments for re-assembly). 


#### Subsequent Update and Multi Fragment Items
The subsequent update contains the fields necessary to identify the MRN data item, the order of this fragment among all the fragments for this item, and the fragment itself. The other point to note is that (for a Multi fragment item), Update messages with FRAG_NUM >1 will have fewer FIDs as the metadata is included in the first Update message (FRAG_NUM=1) for that item

#### News Fragments simple handle logic


<img src="images/mrn_flow_reconstruct.png"/>


In [9]:
def processMRNUpdate(ws, message_json):  # process incoming News Update messages

    fields_data = message_json["Fields"]
    # Dump the FieldList first (for informational purposes)
    # decodeFieldList(message_json["Fields"])

    # declare variables
    tot_size = 0
    guid = None

    try:
        # Get data for all requried fields
        fragment = base64.b64decode(fields_data["FRAGMENT"])
        frag_num = int(fields_data["FRAG_NUM"])
        guid = fields_data["GUID"]
        mrn_src = fields_data["MRN_SRC"]

        #print("GUID  = %s" % guid)
        #print("FRAG_NUM = %d" % frag_num)
        #print("MRN_SRC = %s" % mrn_src)

        if frag_num > 1:  # We are now processing more than one part of an envelope - retrieve the current details
            guid_index = next((index for (index, d) in enumerate(
                _news_envelopes) if d["guid"] == guid), None)
            envelop = _news_envelopes[guid_index]
            if envelop and envelop["data"]["mrn_src"] == mrn_src and frag_num == envelop["data"]["frag_num"] + 1:
                print("process multiple fragments for guid %s" %
                      envelop["guid"])

                #print("fragment before merge = %d" % len(envelop["data"]["fragment"]))

                # Merge incoming data to existing news envelop and getting FRAGMENT and TOT_SIZE data to local variables
                fragment = envelop["data"]["fragment"] = envelop["data"]["fragment"] + fragment
                envelop["data"]["frag_num"] = frag_num
                tot_size = envelop["data"]["tot_size"]
                print("TOT_SIZE = %d" % tot_size)
                print("Current FRAGMENT length = %d" % len(fragment))

                # The multiple fragments news are not completed, waiting.
                if tot_size != len(fragment):
                    return None
                # The multiple fragments news are completed, delete assoiclate GUID envelop
                elif tot_size == len(fragment):
                    del _news_envelopes[guid_index]
            else:
                print("Error: Cannot find fragment for GUID %s with matching FRAG_NUM or MRN_SRC %s" % (
                    guid, mrn_src))
                return None
        else:  # FRAG_NUM = 1 The first fragment
            tot_size = int(fields_data["TOT_SIZE"])
            print("FRAGMENT length = %d" % len(fragment))
            # The fragment news is not completed, waiting and add this news data to envelop object.
            if tot_size != len(fragment):
                print("Add new fragments to news envelop for guid %s" % guid)
                _news_envelopes.append({  # the envelop object is a Python dictionary with GUID as a key and other fields are data
                    "guid": guid,
                    "data": {
                        "fragment": fragment,
                        "mrn_src": mrn_src,
                        "frag_num": frag_num,
                        "tot_size": tot_size
                    }
                })
                return None

        # News Fragment(s) completed, decompress and print data as JSON to console
        if tot_size == len(fragment):
            print("decompress News FRAGMENT(s) for GUID  %s" % guid)
            decompressed_data = zlib.decompress(fragment, zlib.MAX_WBITS | 32)
            
            json_news = json.loads(decompressed_data)
            _news_messages.append(json_news)
            print("News = %s" % json_news)

    except KeyError as keyerror:
        print('KeyError exception: ', keyerror)
    except IndexError as indexerror:
        print('IndexError exception: ', indexerror)
    except binascii.Error as b64error:
        print('base64 decoding exception:', b64error)
    except zlib.error as error:
        print('zlib decompressing exception: ', error)
    # Some console environments like Windows may encounter this unicode display as a limitation of OS
    except UnicodeEncodeError as encodeerror:
        print("UnicodeEncodeError exception. Cannot decode unicode character for %s in this enviroment: " %
              guid, encodeerror)
    except Exception as e:
        print('exception: ', sys.exc_info()[0])

### JSON-OMM Process functions

In [10]:
def process_message(ws, message_json):
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if "Domain" in message_json:
            message_domain = message_json["Domain"]
            if message_domain == "Login":
                process_login_response(ws, message_json)
            elif message_domain:
                processRefresh(ws, message_json)
    elif message_type == "Update":
        if "Domain" in message_json and message_json["Domain"] == mrn_domain:
            processMRNUpdate(ws, message_json)
    elif message_type == "Status":
        processStatus(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))


def process_login_response(ws, message_json):
    """ Send item request """
    send_mrn_request(ws)


def send_login_request(ws, refresh):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': 1,
        "Domain": 'Login',
        'Key': {
            #'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            },
            'NameType': 'AuthnToken'
        }
    }

    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position
    login_json['Key']['Elements']['AuthenticationToken'] = token
    
    if (refresh == True):
        login_json['Refresh'] = False

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

### WebSocket Process functions

In [ ]:
def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws, False)

def create_socket(address):
    login(None, None)
    # Start websocket handshake
    print("Connecting to WebSocket " + address + " ...")
    web_socket_app = websocket.WebSocketApp(address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()
    return web_socket_app

if __name__ == "__main__":
    ws_address = "wss://{}:{}/WebSocket".format(hostname, port)

    try:
        web_socket_app = create_socket(ws_address)
        time.sleep(10)

        while True:
            if web_socket_open == True:
                # When connection is opened, check for token refresh
                time.sleep(sleep_for_seconds)
                counter = counter + sleep_for_seconds
                #print('Counter', counter)
                if (counter % 250 == 0):
                    login(web_socket_app, token_refresh)
            else:
                # When connection is closed, reconnect
                counter = 0
                web_socket_app = create_socket(ws_address)
    except KeyboardInterrupt:
        web_socket_app.close()

Retrieve Token
Connecting to WebSocket wss://apac-3.pricing.streaming.edp.thomsonreuters.com:443/WebSocket ...
WebSocket successfully connected!
SENT:
{
  "Domain":"Login",
  "ID":1,
  "Key":{
    "Elements":{
      "ApplicationId":"256",
      "AuthenticationToken":"eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6InZzUWg1LWI5NmF2aDlueE9rajRVZ2lIaUp0aE9IZ1YzSDg2X2JZUEs5NVkifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwibnNBSlh0VGpFLWFYZ1YzR3VUd2picHdLQWpFU2dldG1mdWxkZWhkNjU4M29ySGtMQ1NPUFMwQW1RVm1NVUtlT05PS3pZcVFmZVo1V2d2azI1bUkteF9DZkxSeEctaDdkb0s3S3kzNk5iNXVYeDJtMXAzazNqX0U3SlByNFhzcW5XU1NOWGpnZk0yNVQ4NzcyVUx5NW9XNnY5ek0yLTJRZUlKNlA1RHgwUEc5MnhBMklyMWVtVVdlOFE2MHFKYl80N2ViTW1GVExVMjYtUmh6NG05RGh4TGw1QUhEenY3Njl5R2haeFFMYXdWa1MwQ1M1NEgxT09ITDdicnVBSVdhSUVNMC1qc1c2NkZKc2dTVE80S0R1VWxZS0FPU0hUZWJLNzZSVFlIdDhfYlRwa3k3bGVsOVZwYndpenlpR1NJRDhpaFFnSDk2R1NramwwZFNvS0ptbi1PZVpRUXdTT212VjBPN0JoVXdWT0U5U28zdXpWS3QxcVdERFRPbFJSa2Q5bHBaQ1VPaUl1NzBQdzVGYlJqamxDaldCQU9hUDEtdXBleUpjLTI0eDFSeS10a081aHdLUXduM

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/21UbU8aQRD+3p9xn30BrCj3jagloqItplVrYw45LAhIucPUNk1uD1GJL8R3rS/E+IbQINUqYKj9MevewSf+QmcXjDZpcpmdnX12npnZmfvKCX65283xXLDHN2G1vvNxDZwQcXvF4Lgocfx7zjHA9wx39oOZaXXljd3BfWjgXFPuGbg7GsSKop8lyNURKagYZSuZHXJ9S7LbDwWlEl0BY2Uzj9FltbRaLW1p26dYQeX5tJZbJbHvgC8nYvr9BUFxPZ8oHyOMlrStHEZJjNKVsy19r4BRpoIKZP4Oo02sLrGjdYxSYMfK0jPqS3ZxG6NZHF3HagFHl7FaBDqs3uBoiRmLWF0j8RusLmM0x5wA/gyjZepWUSEYLZeAGKqlBW0lVVGugIX8LuqZRYxOqqV4GegWbrWNYuXgAlxpKwBYYx5msQqYLIRB/kRJKU8tCrhNaftJch/TZpOA10/uGCPlgpjJXAyj83Jqp6zEyN4xcJXThxgd1wCjwWfF1fch8yzJJUj+QPu1qCdPH+6A7xJKQPZzJAlFAcABK9Ahk1k9XizHryjrBpwuMuI0halxEotqOdhm9N078nMTo10Wfa2scPecVkRdhNfS9n+AB0geowTFqAu0cAqCJKlPNV5Lr+6BJUyroF7j6BGOglygFZ9/SptmBY3k8YYluSMsCrJIW9BkMBkaDZZGo3nQ0M7DZ2xtMpgtI4D8KApuvzcoAgqe57Ensv+8BIvnsehP7QidBw68lKDe4mNjYyaDwWI0t5omXw3OuFpEsyUQ6POL/ZLd6mkOv+3r/O

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/5VW224bNxB971cM9NQkkry7kixpURRVFDs26siuZTdpmsLYCyUxWpEbkmtFLQr0vV+S78if9Et6hlSca9EWMCwuhxzOnDlzyN9aWeVOy1baUo+vLw935Xw7arVbWVNKoQphW+nPrdlFClvrl3Yr1+UOS1+oaSWLNa2EEeQ0WSFIN4aKVWZcSi/UC7VyrrbpwYGfsl1nslKqZSEURlW30Ju95WCjxeu8ME2eRNE4PoyTaBwNuku5YC/ncFrLW+2o1lI5kpYyR/E4Go66UdS9W2LEAqEg3pQqrde00IYyKo2uEd42M6UNmygXMAnYct1gufcwqZwwKnPyVpBFhJmROiW3ElTqrbKyFJQbka1140gv9o62uqlKKrKq8ofFo8FoQJkqMUrGPe93qjcb5BtcXc5PffS5xinSWVKiYSik21EG75zXIOrSFdY+mUwf8WIlliEq9puLSm/9TiuXKquokkqE9bVQAr6c1IoDHESEwbtDPwl00RhYzF1qXXpyfvSMHlaNoOlK1nSqSvGaT9+XjBpMGH9wElGZ7RAefR2PB9G4G/XvUQ5U3meFs2sB/Mr9ql48vufBmDd1rVFvn4sRVlqXMb9SNiZxnPRQULp/339GwNB/hq9Br/+hMekN8cnjeNwfjbv9IVWZdWEiGUZ3xsCS/T5foTunoUp3xk6nwz9Xl5NHp7PHND2aYXjmC0ZFKGNmduC5uZUFWFKLQqJ2lvGRwFoUKyWBMbLLqh2S8zkfA2+bVcKSUK8aaSRGdSUyK0I9dtwzl+CBMDioKEBJRxu4WAJw7L

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/71XbW/bNhD+vl9B+MPQYo4jWXFehHWDIsuJkFhxJdnJOg8FLdEWW5lUSSqOM+y/70gpTtpt3VqsEZzcibqX54534un3Di5VmHfcDkuC1F8fjTe40+3gOqeEZUR23F870cT1oxhWG8ZrOBDv/NbtLHi+Be2YVFwoxBlSBUGCyLpUEvElkrgkhpKsFlRRItGL0cDZG7ycszlr1XKsPhXCZckzrChY1JeL7MO9hFR7fatvzdkYKy24bTQ/c4He4InenA2vkdyuF7xE/3K5yBvOPNuZ9C3rxJuzqF4viDAo64XMBF2QHIG1FzWj6uVHitaXubn2bcf/1l4C7xmcnE0vR62b02/oJpxdPkMwF37wVV6Ore7A+jJX4/Q5SiDxrp+jCBLffy4v37w303gafG0wc+bVquCC3oOU4kjSFdMvyAUpcLnUVvSrspVH47iH/PPwHO2h82l0hi7DaM4yvq4w287ZhEu6exd+HjIYCUboahLEXhqCoavRKPSD+IvQjOOkh5Kpf+4NPYSSq2F6Hp6ee5OL6eXXgQpuAn+ahrMADcM48NOrWOfn7f9yzVlaUDgwGOM1HFlrwhTaYIkqQSosIFrMcpRTKcmaMjgucrTYopJKzbWxIC4QCNSwuXOmCsHrVWHyQUqSKcEZzaBkQGONNgXNChAG8/yW5mBjCcpatqpFxaU5xx69QXrmrE0uZSC6bs40jUmQ0sDJeVZr1Oa8/EdgsGdgZM4SxbP3KLjLCsxWxltaYFpqg5yV2x6Ce4I+I1VAbhBifM7gmK44k3RBS32WPgSScS

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/71XbXPaRhD+3l9xw4dOMoOxBAYbTdOOLIRRbWQiCeykdDKHdKCLxR25O4Fpp/+9eycZO2mbNpnGGuxdnfbl2b1d3er3Bi5UkDWcBov9JLvYfjjBjWYDlxklLCWy4fzSCCeOF0awWjFuxYF449dmY8GzPWhHZMOFQpwhlRMkiCwLJRFfIokLYihJS0EVJRK9GHY7R92XczZntVqG1adCuCh4ihUFi/pykN07isnmqG21rTkbY6UF95XmZy7Q6z7Rm7PBDZL79YIX6F8uB7mDmWt3Jm3L6rtzFpbrBREGZbmQqaALkiGw9qJkVL38SNH6Mjc3nt3xvrUX330GJxfTq2Ht5vwbuglmV88QzKXnf5WXM6vZtb7M1Th5jhKI3ZvnKILY857LyzfvzSSa+l8bzJy5pcq5oL+BlOJI0hXTL8gFyXGx1Fb0q7KWR+OohbxRMEJHaDQNL9BVEM5ZytcbzPZzNuGSHt6Fn4cMRvwhup74kZsEYOh6OAw8P/oiNOMobqF46o3cgYtQfD1IRsH5yJ1cTq++DpR/63vTJJj5aBBEvpdcRzo/7/6Xa86SnMKBwRgv4chaE6bQDku0EWSDBUSLWYYyKiVZUwbHRYYWe1RQqbk6FsQFAoESNnfOVC54ucpNPkhBUiU4oymUDGis0S6naQ7CYJ5vaQY2lqCsZTel2HBpzrFHb5CeOauTSxmIrqszTWMSpDBwMp6WGrU5L/8RGOwZGJmzWPH0Dvn3aY7ZynhLckwLbZCzYt9CcE/QZ6RyyA1CjM8ZHNMbziRd0EKfpQ+BpF

...

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/41RTVPbMBC991d4dE5sS2lIrZtTAk0hjhubYUKn05Fjkai2ZVcfBKbTv8GRK9fwGxL+F6vCcO5pd5+edvft+4NYbaYlokhOz+Li4za6PEY9xGwpuFxxjeh3lKR0miXoRw8VbXkH1I0xnaZBsN1u/Yp1fqvWvlHBWNSlUKIJPo2GZIShzbVQ2nxWnBnuRpCQhP0w6uOjHIcUExpGPh4OroC54aysheTAOuyyPE7GF+deOl/kJ/v7pbfc3yeT/LCbHXZe7D8/+F7fO+WS1168fzw8VTVrGDQRbsi7jJ8kDCN8RKKvhJE5S6tR0SQSF+mQRV02GSxXm6rNblfTjS3G7rfUhjnN82tQDWprJteWrd1ORgGh4VpDmd91ABGoRfNWIMNvTdDVTEjgdaq9ESVXgCcZPYtTh9kiM8xYOCiCMYZazYqaw4u2xS++Mv8uPaP4mzv/jA7Ia/gCISH0PF+8Jsnk8i1bTE6dJYZVPOO/rXMLUdxDVq0hB5sGPXTDlRat/A8H/n54AU1E4cQKAgAA",
      "FRAG_NUM":1,
      "GUID":"IKAb4w9WD_20091629J2a2OaPk7bmNn1bP5a9pSE3YchkoSxcIhubB",
      "MRN_SRC":"HK1_PRD_A",
      "MRN_TYPE":"STORY",
      "MRN_V_MAJ":"2",
      "MRN_V_MIN":"10",
      "TIMACT_MS":36729270,
      "TOT_SIZE":408
    },
    "ID":2,
  

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/41TTW+bQBC991cgDjnZGDDgsFJUET4CCWAKdpS0qips1vY2wBJY7ERR/3tngebcy7w3s7Mzjx3mQ8xLFhQiEmt3HWqn1/uDJs7EvC8Irve4E9EPMU4QnIk/Z+KOFu+QemKsQYvFgZSkPnYS2ecSqZnETrTqaN3inuG2k/a0+kw5E3zB7cKhl7qkeSHlXfP21aH7vsI1tL/RdU02Vyv1yqY1g5BH2ypnN01xuLKapoQOjNAaEl3yQusYXzqrxC2LaE0YbaHFIzQA3QfSdsxucc4w/yZVVuW5bM4VY6PISFGRIkvXuv4dMk84L0AbhqzUfbL8WEh8K40s291uAtsKs5kQxLYkeKTEnWCBzoJrFRgVUnwkHWsHSULGoNdwAg6XLcznQjbXFhY0IVzE57v+UmXZVAy1r6irVk2WnJ6a5uHxzM4PanYJn/UY2wfjTF8r1X/kt+uO5XwI6wOMAZ6/zOtjnx+5ZlxDQoW7DtzNewMhFXxSTY7I8BtbNGVOeF7T0jMpcAvxOEOuc8dj/Y5L72HCIrRhqO/yXYnhpOt3v/GeDaO/RYp2DTFA3ZjQHHG1XAFxkTJYQx7gfrApWM9HSw4hUsyBREjlF+6QNljjfgBeLELKN2fAeDsAn0+KYCyxtAYaq8iK3DQb6a1lbyYWrJ30buR2lMTPI3VcK1RG6gVhEE8ZXppN97zMtbdTNd+1rZEFsbMdWejGI4lS2//HpirRJnG8kcZc1EgTP5pYpo041E+Qppq6KauKseTrw/IXnOHXnm+WiJSZ2LdH4LBSy5l4hp2BH+

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/51W224bNxB971cM9JSLJO9FsqVFUVRQHFuoI7uS3CZtCmO1S61or8gNybWsFAX63i/Jd+RP+iU9w/Ult15QwLCWHHI4c+bMIX9tpaWb5K2kpY7OZ/3pPKpetdqttM6lUJmwreTn1vQsga31S7u11PkOS1+rcSmzK1oLI8hpskKQrg1l69S4hF6r12rtXGWTvT0/ZbvOpLlURSYUvspupje3lr21uxG1wV8UBMNwP4zCMDjoFnLFXk7htJLX2lGlpXIkLaWO4rjf7wZB936FEStEgnATcmtBud4qK3PBhutUlthkqdSq4F/ezH6MsNK6FJu8n1HphFGpk9eCLMJMjdQJpUuNMW9pw4G+opU2FPd6MaUqx8dw328e680GmTWnz+YT9r8Upd5SP+jSApMvRuNnD7PSWbKyUGlJpVTCO1Oi8Kc36zOtVrKoDaa04o0P5uNaFYguVbQAqCKnw/MZSZWLG+J88g8PiQLvux9QJZBQjjjoETKvRMbOyl0D537cHUS3KcWDbtx/7NOa11WlUSNvecDLJmz0yQcBPXnSjHqxH/lBU587UxwFGPnPCIZwSGVqnR+HvcGdKfjIXdDr3buLhsPowdTpdPhnMRs9m0yPaHw4xeeJZwZQ83VIzQ6UNNcyQy2RqkxLacE+oEROZGslMyCfAv4dcvKpPkddbVoKS0K9qaWR+KpKkVqUIi1L2jG9Z6IGSXBQlukabNzARSEMYe99OA3B2eVyR9VaK3CSpmJLr7S5okfhY4rCiAa9A4riwUGbTrTKUeFHT3s9mIKYer2ADv

Counter 90
RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/41WwXLiRhC95yumdNqtYr0Ib7w2NzDYS2wwBryubEilRqhBs0gjdmYEMan8SY7+hpx848fyeiStSU4pqjSjoaf79evXM/ojkKkbxEE70J3H8cf7p3HTBI1AFrEivSAbtH8JRuM2/gt+bQRRHj/BdK7fDCmNC70ScWEWyYoS0rEgszo86717O9f865LR4o3cbd7bWL4V78SkgJGmQothrqUjoeUiETEZ0YnsIgEQ9qisE7G0ouPybG3k0u3IrOd6eHhJUooQAzuWSiuntmIOoNbCQZecURTNgxPRU1S/2oh2Kk3VqvZLJs0RifRJCbH33zgikREJmaYkPufGO3d77CYtlocXI8CKQKqrw0vq1IrmuuPjR2U8LZOMhM13sCrNse/wF4OWjECX+41F7lMHZCkxNLtJpIUvMtiUpq4hHKkUNkxNH8srRetc75nP7s2jkJkYKud2OdjLlDsRX4ptbsTGHF6WvEvaue6rGKAPf1urOGExKtYpSWMVXkxCylml7YbWDlk60R9NByU2hVwNwcoR+HnQDpRJTp9rB3iJcd6KxKDcr3ItVCYembzjGrEuKpYH2hORgkp4rYqvxa4wvIzNE5IMZEXR4bmISQyltZ5LLfYFKFDaMCJQBZIQ2wCNjBKIbaUY2w3H9WokK7qGtKaUsNsBy2JN2lY12SFvZCc6KJjyCYI3VKas3ndVTPMaG8zsRlFqsdOSZ2jqqOAKHp4jajD9nBtnc3hODHeBx1Dpj3fM9S02uxpelasVK7LelSuDNSqMqDm8dXQJSUC

RECEIVED: 
[
  {
    "Type":"Ping"
  }
]
SENT:
{
  "Type":"Pong"
}
RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "DoNotRipple":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-09-16",
      "FRAGMENT":"H4sIAAAAAAAC/41W2W7bRhR971dc6ClpJJmkJEsiiqKqvAl1ZEeyszWFQZEjamJqhpkZWlaKAn3vl+Q78if9kp47lJ21bQDD4mx3OffcM/N7IyncJGvEDXV8OUv7g5fzotFsJFUmhUqFbcS/NqbnMdYavzUbC51tsfWVGhcyvaaVMIKcJisE6cpQukqMi+mVeqVWzpU23tvzU7btTJJJladC4atop3q9W9kz2a2oDP6iIBiG+2EUBlG3ncslWwmjfq8dBCStt1/KG+2o1FK5Ni+f8ZwRS4SBWGOyK20cOWHWZMRCVypDdJvEZJbCbtCHJX9sVGCLSpy8EWQRUmKkjmkhCr2hnccmidtSpA7jaMARJLAVRkF4Z2Os12skE5NbCZrNJxxibaEXtOkCk49H4wOeTRYafqSzZGWukoIKqYS3p0Tug6j3p1otZV4ZTGnFB9fyVmRtmlXWykTRgSi1lS4xWzq8nJFUmbgl62DozjP7iOpQewGVAmllCIMeGGE5GbgqtpQgqU4YtjvDOqlOd9CO+g99VvOqLAGh9Ss4Jdk+WBD7YnT3I4bi++/rUafrR/XAo3u31OkPMao/4SmMqEisqws67Nwt7Wq7O7PDeTfwQO+WWq0W/1zMRgeT6TGND6f4PPXQAjRfBgbFCnMjU1QUqcqkkBZ8A0qgQ7pSMgXwCdDfIief6pE2ZJNCWBLqTSWNxFdZiMSiEklR0JYJNxMVqAJHaQo2OVrDRC4

### Real-time News Data Model
The news data appears as JSON in UTF-8 after decompression and assembly of the individual messages.
The Real-time News feed contains the headline, story body text, and associated metadata about the story as a simple group of named values.

Example JSON fields are following:
- ```firstCreated```: UTC timestamp for the first version of the story. Millisecond precision.
- ```headline```: The headline text of the news item.
- ```id```: Uniquely identifies the news item. This is the same value as the GUID in the OMM envelope.
- ```body```: The full body text of the news item (Story Body).

For more detail regarding MRN data model please visit [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item) page. If you want to compare MRN and the legacy N2_UBMS news data model, please visit [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).



In [11]:
print("first 10 headlines\n")
for news in _news_messages[:10]:
    if news["headline"]:
        print(news["headline"])

first 10 headlines

Shenwan Hongyuan Group Co Ltd - 2019 INTERIM REPORT
PDF 1: PALADIN ENERGY LTD (Share Purchase Plan Offer Document)
PDF 1: PALADIN ENERGY LTD (Share Purchase Plan Offer Document)
BREXIT-ZEW SAYS THERE IS STILL HOPE THAT A NO DEAL BREXIT CAN BE AVOIDED
HNX - CKV_20190917155723_Result of transactions of Directors PDMR Ly Chi Duc
HNX - GKM_20190917155723_Board Resolution
Announcement
إعلان
為替相場（対ドル）
삼성 QLED vs LG OLED…불붙는 `8K TV 기술전쟁`


In [12]:
print("first 10 stories\n")
for news in _news_messages[:10]:
    if news["body"]:
        print(news["body"])

first 10 stories

2019 INTERIM REPORT
For best results when printing this announcement, please click on link below:
http://pdf.reuters.com/pdfnews/pdfnews.asp?i=43059c3bf0e37541&u=urn:newsml:reuters.com:20190917:nSN6tXcL2


Please click the above URL to view the article.
Latest version of Adobe Acrobat reader is recommended to view PDF files.  The latest version of the reader can be obtained from http://www.adobe.com/products/acrobat/readstep2.html.

                             PALADIN ENERGY LTD
                                        ACN 061 681 098

17 September 2019


ASX Market Announcements                                              By Electronic Lodgement
Australian Securities Exchange
20 Bridge Street
SYDNEY NSW 2000



NOT FOR RELEASE TO U.S. WIRE SERVICES OR DISTRIBUTION IN THE UNITED STATES.

                         SHARE PURCHASE PLAN OFFER DOCUMENT
Paladin Energy Limited’s (ASX:PDN) (“Paladin” or the “Company”) provides the attached offer
documentation in respect of it

In [13]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install pandas and numpy packages in a current Jupyter kernal\n

# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install numpy

In [14]:
import pandas as pd

In [15]:
df_headlines = pd.DataFrame(_news_messages, columns = ["firstCreated","messageType","provider","language","headline"])

In [16]:
messageType = {
    0: "Unknow",
    1: "Alert",
    2: "First take",
    3: "Subsequent take",
    4: "Correction",
    5: "Corrected",
    6: "Update",
    7: "Deletion",
    8: "Drop due to expiry"
}

In [17]:
df_headlines["messageType"] = [messageType[value] for value in df_headlines["messageType"] if value in messageType ]

In [18]:
df_headlines

,firstCreated,messageType,provider,language,headline
0,2019-09-17T09:01:42.835Z,First take,NS:HIIS,en,Shenwan Hongyuan Group Co Ltd - 2019 INTERIM R...
1,2019-09-17T09:01:42.841Z,First take,NS:SGX,en,PDF 1: PALADIN ENERGY LTD (Share Purchase Plan...
2,2019-09-17T09:01:42.833Z,First take,NS:SGX,en,PDF 1: PALADIN ENERGY LTD (Share Purchase Plan...
3,2019-09-17T09:00:08.808Z,Alert,NS:RTRS,en,BREXIT-ZEW SAYS THERE IS STILL HOPE THAT A NO ...
4,2019-09-17T09:00:41.514Z,First take,NS:HNX,en,HNX - CKV_20190917155723_Result of transaction...
5,2019-09-17T09:00:45.253Z,First take,NS:HNX,en,HNX - GKM_20190917155723_Board Resolution
6,2019-09-17T09:01:37.161Z,First take,NS:BHB,en,Announcement
7,2019-09-17T09:01:38.520Z,First take,NS:BHB,ar,إعلان
8,2019-09-16T15:01:25.665Z,Corrected,NS:KYDO,ja,為替相場（対ドル）
9,2019-09-17T09:01:52.643Z,First take,NS:MKN,ko,삼성 QLED vs LG OLED…불붙는 `8K TV 기술전쟁`


In [19]:
df_story = pd.DataFrame(_news_messages, columns = ["headline","body"])

In [20]:
df_story

,headline,body
0,Shenwan Hongyuan Group Co Ltd - 2019 INTERIM R...,2019 INTERIM REPORT
1,PDF 1: PALADIN ENERGY LTD (Share Purchase Plan...,For best results when printing this announceme...
2,PDF 1: PALADIN ENERGY LTD (Share Purchase Plan...,PALADIN ENERGY LT...
3,BREXIT-ZEW SAYS THERE IS STILL HOPE THAT A NO ...,
4,HNX - CKV_20190917155723_Result of transaction...,\r\n\r\n Double click on the URL below to view...
5,HNX - GKM_20190917155723_Board Resolution,\r\n\r\n Double click on the URL below to view...
6,Announcement,([1]Arabic) ([2]English)\n\nReferences\n\n ...
7,إعلان,([1]عربي) ( [2]إنجليزي)\n\nReferences\n\n ...
8,為替相場（対ドル）,\n１７日\n\n（日本時間）\n\n ０時 １０８円０１－０２銭\n\n １時 １０７円９...
9,삼성 QLED vs LG OLED…불붙는 `8K TV 기술전쟁`,\n 삼성전자와 LG전자가 8K(4K·UHD의 4배 화질) TV의 화질 등 핵심...


## References
* [Refinitiv Elektron SDK Family page](https://developers.refinitiv.com/elektron) on the [Refinitiv Developer Community](https://developers.thomsonreuters.com/) web site.
* [Refinitiv Elektron WebSocket API page](https://developers.refinitiv.com/websocket-api).
* [Developer Webinar Recording: Introduction to Electron WebSocket API](https://www.youtube.com/watch?v=CDKWMsIQfaw).
* [Introduction to Machine Readable News with Elektron WebSocket API](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor](https://github.com/Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor).

For any question related to this example or Elektron WebSocket API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com/spaces/152/websocket-api.html).